In [13]:
import pandas as pd
import altair as alt

# Load your dataset
data = pd.read_csv("NBA_Players.csv")  # Replace with your dataset

# Data Preparation
data['total_points'] = data['pts'] * data['gp']  # Total points = avg points * games played
data['total_rebounds'] = data['reb'] * data['gp']  # Total rebounds = avg rebounds * games played
data['total_assists'] = data['ast'] * data['gp']  # Total assists = avg assists * games played

# Group by team and season to calculate total metrics
team_season_data = data.groupby(['team_abbreviation', 'season']).agg(
    total_points=('total_points', 'sum'),
    total_rebounds=('total_rebounds', 'sum'),
    total_assists=('total_assists', 'sum')
).reset_index()

# Create a selector for the team
team_dropdown = alt.binding_select(options=sorted(team_season_data['team_abbreviation'].unique()), name="Select Team:")
team_selection = alt.selection_point(fields=['team_abbreviation'], bind=team_dropdown, value=sorted(team_season_data['team_abbreviation'].unique())[0])

# Melt the data for easier Altair plotting
team_season_data_melted = team_season_data.melt(
    id_vars=['team_abbreviation', 'season'],
    value_vars=['total_points', 'total_rebounds', 'total_assists'],
    var_name='Metric',
    value_name='Total'
)

# Create the line chart
chart = (
    alt.Chart(team_season_data_melted)
    .transform_filter(team_selection)
    .mark_line(point=True)
    .encode(
        x=alt.X('season:O', title='Season', axis=alt.Axis(labelAngle=-45)),
        y=alt.Y('Total:Q', title='Total Metrics'),
        color=alt.Color('Metric:N', scale=alt.Scale(domain=['total_points', 'total_rebounds', 'total_assists'], range=['steelblue', 'orange', 'green']), title="Metrics"),
        tooltip=['team_abbreviation:N', 'season:N', 'Metric:N', 'Total:Q']
    )
    .properties(
        title="Metrics Across Seasons: Total points, rebounds, and assists",
        width=700,
        height=400,
        padding=0  # Remove internal padding
    )
    .configure_view(
        continuousWidth=700,
        continuousHeight=400
    )
    .add_params(team_selection)
)


chart.save("nba_team.html", format="html")
chart

alt.Chart(...)